In [ ]:
import glob
import csv
import pandas as pd
import shutil
import os

In [ ]:
files=glob.glob("/content/drive/My Drive/IBM_sentiment_analysis/data/hydrated_csv_files/*.csv")

In [ ]:
#This function extracts the hastags from the hastags string 
def text(x):
  f=[]
  x=eval(x)
  for i in x:
    f.append(i['text'])
  return f

In [ ]:
sum=0
for f in files:
  df=pd.read_csv(f,header=None)
  df1=df.iloc[:,[0,2,3,4,9,10]]

  # converting Created_at twitter value to python datetime format and extracts only date to date column
  df1['date']=pd.to_datetime(df1[0]).dt.date
  df2=df1.iloc[:,[6,1,2,3,4,5]]
  df2.columns=['date','id','text','hashtags','retweet_count','language']

  #checking for duplicate IDs
  df3=df2.drop_duplicates(subset='id', keep="first")
  df4=df3[df3['language']=='en']

  #getting hashtags from hastag string
  df4['hashtags']=df4['hashtags'].apply(text)
  df5=df4.iloc[:,[0,1,2,3,4]]

  #Loading geodata files
  geo=pd.read_csv('/content/drive/My Drive/IBM_sentiment_analysis/data/text_files/geo_'+str(df5.iloc[0][0])+'.txt',delimiter='__',header=None)
  geo['f']=geo[0].str.replace(',','//',n=1)
  geo[['id','location']]=geo['f'].str.split("//",expand=True)
  geo1=geo.iloc[:,[2,3]]
  geo1['id']=geo1['id'].astype('str')
  df5['id']=df5['id'].astype('str')
  geo2=geo1.reset_index(drop=True)
  df6=df5.reset_index(drop=True)
  df6['id'] = df6['id'].apply(lambda x: x.strip())
  geo2['id'] = geo2['id'].apply(lambda x: x.strip())

  #Merging the tweets to their geodata
  dff=pd.merge(df6,geo2,on=['id'],how='left')

  #Saving single tweets file 
  dff.to_csv('full.csv',mode='a',header=None,index=False)
  sum=sum+dff.shape[0]
  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

In [ ]:
print(sum)

14878803
